In [121]:
class ReviewEmbedding:
    import pickle
    import os
    import random
    import sqlite3
    import string,numpy
    def review_emb(self,text, word_emb,word_dict):
        words = []
        rev_w = text.lower().strip().split()
        for w in rev_w:
            prev = 0
            for ch in range(len(w)):
                if w[ch] in self.punc:
                    if prev < ch:
                        words += [w[prev:ch]]
                    words += [w[ch]]
                    prev = ch + 1
            if prev < len(w):
                words += [w[prev:]]
        emb = numpy.array([0.0]*50)
        for word in words:
            if word in word_dict:
                emb += word_emb[word_dict[word]]
            else:
                word = 'unk'
                emb += word_emb[word_dict[word]]
        return emb
    
    def Select(self,q):
        return self.c.execute(q)

    def Update(self,q):
        self.c.execute(q)
        self.conn.commit()
    
    def __init__(self):
        self.review_dict = {}
        self.c = None
        self.conn = None
        self.review_dict = None
        self.punc = set(string.punctuation)
    
    def create(self,db_path,word_dict_path, word_emb_path,review_delim_path,review_emb_path):
        
        self.conn = sqlite3.connect(db_path)
        self.c = self.conn.cursor()
        q = self.Select("select review_id, review_text from Reviews ")
        word_dict = pickle.load(open(word_dict_path,"rb"))
        word_emb = pickle.load(open(word_emb_path,"rb"))
        review_dict = {}
        i=0
        review_dict = {}
        prev = 0
        with open(review_emb_path,"w") as f:
            for row in q:
                review_id = row[0]
                emb = self.review_emb(row[1],word_emb,word_dict)
                line = review_id+"\t"+ " ".join([str(x) for x in emb])
                f.write(line+"\n")
                
                review_dict[review_id] = (prev,len(line)+1)
                prev += len(line)+2
                if i%100000==0:
                    print(i,end=" ")
#                     break
                i+=1
        pickle.dump(review_dict,open(review_delim_path,"wb"))
#         del word_dict , word_emb
        self.conn.close()
    
    def load(self,review_delim_path,review_emb_path):
        self.review_dict = pickle.load(open(review_delim_path,"rb"))
        self.review_emb_path = review_emb_path
    
    def get_emb(self,review_id):
        if not self.review_dict:
            raise Exception("load the word index and emb files")
        if not review_id in self.review_dict:
            return None
        start,length = self.review_dict[review_id]
        with open(self.review_emb_path,"r") as f:
            f.seek(start)
            line = f.read(length)
#             print(line)
            _id , emb = line.split("\t")
            emb = numpy.array([float(x) for x in emb.split()])
        return emb
    
    




In [122]:
re = ReviewEmbedding()

In [123]:
re.create("yelp_dataset.db","word_dict.pkl","word_emb.pkl","review_delim.pkl","review_emb.txt")

0 100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 1100000 1200000 1300000 1400000 1500000 1600000 1700000 1800000 1900000 2000000 2100000 2200000 2300000 2400000 2500000 2600000 2700000 2800000 2900000 3000000 3100000 3200000 3300000 3400000 3500000 3600000 3700000 3800000 3900000 4000000 4100000 4200000 4300000 4400000 4500000 4600000 4700000 

In [116]:
re = ReviewEmbedding()
re.load("review_delim.pkl","review_emb.txt")

In [117]:
re.get_emb("3zRpneRKDsOPq92tq7ybAA")

3zRpneRKDsOPq92tq7ybAA	10.8397502047 -20.7875054753 -21.0155165261 5.30395240127 20.9272409028 18.6491672242 -1.71042521793 -6.01856238721 14.1530407223 2.77088434121 -10.1481847508 -7.72026975534 -22.6817526534 -12.2489046904 -20.9182886333 -15.4042131794 -12.8739968382 17.5563024734 -8.27728113794 16.8488769494 -28.4643668646 -8.67103013274 -25.3974421099 -1.73288164008 -4.3695127297 2.2759243805 9.89452796231 12.807304951 -25.1188206952 16.6957012216 -33.2521268144 21.074094441 -19.3086999207 -11.2590874318 15.553153472 15.9926440089 -23.6293763213 -4.21477821528 0.369510335615 -24.7031148898 -10.629140518 5.94363526255 17.0216949977 -14.3600980202 0.73590193782 2.35386800262 2.97065577761 9.86721958534 0.580517903698 4.92248448404



('3zRpneRKDsOPq92tq7ybAA',
 array([ 10.8397502 , -20.78750548, -21.01551653,   5.3039524 ,
         20.9272409 ,  18.64916722,  -1.71042522,  -6.01856239,
         14.15304072,   2.77088434, -10.14818475,  -7.72026976,
        -22.68175265, -12.24890469, -20.91828863, -15.40421318,
        -12.87399684,  17.55630247,  -8.27728114,  16.84887695,
        -28.46436686,  -8.67103013, -25.39744211,  -1.73288164,
         -4.36951273,   2.27592438,   9.89452796,  12.80730495,
        -25.1188207 ,  16.69570122, -33.25212681,  21.07409444,
        -19.30869992, -11.25908743,  15.55315347,  15.99264401,
        -23.62937632,  -4.21477822,   0.36951034, -24.70311489,
        -10.62914052,   5.94363526,  17.021695  , -14.36009802,
          0.73590194,   2.353868  ,   2.97065578,   9.86721959,
          0.5805179 ,   4.92248448]))

In [118]:
re.get_emb("DuffS87NaSMDmIfluvT83g")

DuffS87NaSMDmIfluvT83g	2.49910431867 -6.20021877158 -5.76443344646 0.639091578778 3.66573869984 6.23046622588 -0.626430550823 -2.23511670064 2.45080363518 1.00042104675 -1.3470437238 -1.27315407153 -6.97422029078 -3.64196982235 -5.82523877639 -3.36901434802 -1.97984313872 3.64175202977 -2.5096348488 3.79745424492 -7.28001698758 -1.49519250123 -8.32157619437 -0.150800594361 -1.12478005141 -1.37509233411 3.1791067468 3.4258258433 -4.67352093896 2.6078904029 -7.41971458262 4.48980853707 -4.08591793757 -3.87533691339 5.48887755536 4.38560517016 -5.82064083661 -1.32745900005 -0.993974504177 -5.91906889181 -1.47143243346 1.44681468606 4.4194191792 -4.2215874847 -0.610602375935 0.0582207120024 -0.409982537851 3.18452279957 -0.677900232375 -0.170240857638



('DuffS87NaSMDmIfluvT83g',
 array([ 2.49910432, -6.20021877, -5.76443345,  0.63909158,  3.6657387 ,
         6.23046623, -0.62643055, -2.2351167 ,  2.45080364,  1.00042105,
        -1.34704372, -1.27315407, -6.97422029, -3.64196982, -5.82523878,
        -3.36901435, -1.97984314,  3.64175203, -2.50963485,  3.79745424,
        -7.28001699, -1.4951925 , -8.32157619, -0.15080059, -1.12478005,
        -1.37509233,  3.17910675,  3.42582584, -4.67352094,  2.6078904 ,
        -7.41971458,  4.48980854, -4.08591794, -3.87533691,  5.48887756,
         4.38560517, -5.82064084, -1.327459  , -0.9939745 , -5.91906889,
        -1.47143243,  1.44681469,  4.41941918, -4.22158748, -0.61060238,
         0.05822071, -0.40998254,  3.1845228 , -0.67790023, -0.17024086]))

In [120]:

re.get_emb("OF1ToqGAubsWGri5iad15w")

OF1ToqGAubsWGri5iad15w	16.2797457094 -33.7269081944 -29.7025621177 8.70015689568 30.0465313187 32.5238578517 -3.30181046447 -8.88307969838 27.9449282829 9.82257871254 -21.1767075285 -19.1317934 -35.9928817512 -27.0534298961 -30.9771081123 -17.5178476272 -21.7859711061 29.4708111486 -18.4085592008 21.994854457 -50.0845063424 -9.89535967656 -42.4788471332 -5.64422726748 -0.491530878469 4.5361609488 17.5229463524 18.2700815254 -36.0711379981 21.8723927471 -52.2215634303 34.0892108877 -28.0268958894 -22.7584057536 18.2301310893 19.7020896411 -38.8791101233 -8.63120471241 0.192048181205 -41.0150940007 -14.5340781947 10.973801662 24.3618474743 -25.7656239676 2.03346421267 5.45119324396 5.69084055157 10.407345154 2.70049430247 8.08667315147



('OF1ToqGAubsWGri5iad15w',
 array([ 16.27974571, -33.72690819, -29.70256212,   8.7001569 ,
         30.04653132,  32.52385785,  -3.30181046,  -8.8830797 ,
         27.94492828,   9.82257871, -21.17670753, -19.1317934 ,
        -35.99288175, -27.0534299 , -30.97710811, -17.51784763,
        -21.78597111,  29.47081115, -18.4085592 ,  21.99485446,
        -50.08450634,  -9.89535968, -42.47884713,  -5.64422727,
         -0.49153088,   4.53616095,  17.52294635,  18.27008153,
        -36.071138  ,  21.87239275, -52.22156343,  34.08921089,
        -28.02689589, -22.75840575,  18.23013109,  19.70208964,
        -38.87911012,  -8.63120471,   0.19204818, -41.015094  ,
        -14.53407819,  10.97380166,  24.36184747, -25.76562397,
          2.03346421,   5.45119324,   5.69084055,  10.40734515,
          2.7004943 ,   8.08667315]))

In [65]:
word_dict = pickle.load(open("word_dict.pkl","rb"))
word_emb = pickle.load(open("word_emb.pkl","rb"))

In [70]:
re = ReviewEmbedding()

In [71]:
re.review_emb("Had Continental style breakfast here a short distance from Waverley Station as I had half hour before a train home and wanted to try somewhere new. Motel1One is nearby and a easy two minute walk, slightly uphill towards the castle from the station. On entry it's modern, cosy has tons of cool yet bright pastel and mochilato colours to accommodate your cold wintertime self. Wooden floors, warm rugs, sheepskins on the sofas, slate walls and oak barrelled tables, with slight distinct Scottish themes due to their tartan furnished accessories completes the feel.  I sat in One Lounge, had tea and food self served for well under £10, you know bran, meats cheese and croissant that sort of stuff. With a copy of local newspaper in hand I really needed to eat, stay warm and access wifi, the reasons I came in.  Some hotels in the city give you access to wifi if your a paying guest, but repeated attempts to access their wifi even with staff help, eventually gave access but in German. On going to reception again for additional help the staff were great got the correct website for me, but still no luck.  Hmmm, by the time this was all done I came back to cold tea and didn't really have time now to finish a second cup of hot Earl Grey, so sorry to staff for removing an almost full cup of tea. Great place though nice breakfast and funky decor especially the plasma tv displaying a roasting log fire in reception, but sadly for me no easy access to wifi when badly required.  Maybe next time I will endeavour to try food against technology!",\
              word_emb,word_dict)

array([ 16.27974571, -33.72690819, -29.70256212,   8.7001569 ,
        30.04653132,  32.52385785,  -3.30181046,  -8.8830797 ,
        27.94492828,   9.82257871, -21.17670753, -19.1317934 ,
       -35.99288175, -27.0534299 , -30.97710811, -17.51784763,
       -21.78597111,  29.47081115, -18.4085592 ,  21.99485446,
       -50.08450634,  -9.89535968, -42.47884713,  -5.64422727,
        -0.49153088,   4.53616095,  17.52294635,  18.27008153,
       -36.071138  ,  21.87239275, -52.22156343,  34.08921089,
       -28.02689589, -22.75840575,  18.23013109,  19.70208964,
       -38.87911012,  -8.63120471,   0.19204818, -41.015094  ,
       -14.53407819,  10.97380166,  24.36184747, -25.76562397,
         2.03346421,   5.45119324,   5.69084055,  10.40734515,
         2.7004943 ,   8.08667315])

In [72]:
with open("review_emb.txt") as f:
    i = 0
    for line in f:
        print(line)
        i += 1
        if i == 5:
            break

VfBHSwC5Vz_pbFluy07i9Q	9.47972604109 -23.6212793237 -24.6500816965 1.07845075059 27.5629479948 21.3071304803 -1.67676156125 -7.85581590468 13.8259968876 6.41526031017 -12.7948484794 -9.73227285885 -27.9491593564 -12.4836213142 -25.7159202718 -15.0710781177 -17.846853596 15.5369437 -13.3614174305 16.3309142432 -35.6781664635 -12.8276101737 -29.7959196874 -5.35159125179 -6.3805052191 5.85708889225 15.0408976208 17.8339078726 -27.5427383587 17.2333047721 -40.3137714803 25.9313707802 -18.6700936613 -19.3870256295 17.4423302345 16.2398767384 -26.6333659301 -6.95086142374 -0.617459634785 -26.2009667804 -7.67093283322 4.93188811012 18.4419555155 -17.0688508335 0.690207112362 -0.0112193159293 4.54543076444 11.1342192388 6.66924565895 8.94382931804

3zRpneRKDsOPq92tq7ybAA	10.8397502047 -20.7875054753 -21.0155165261 5.30395240127 20.9272409028 18.6491672242 -1.71042521793 -6.01856238721 14.1530407223 2.77088434121 -10.1481847508 -7.72026975534 -22.6817526534 -12.2489046904 -20.9182886333 -15.404

In [100]:
_str = "Had Continental style breakfast here a short distance from Waverley Station as I had half hour before a train home and wanted to try somewhere new. Motel1One is nearby and a easy two minute walk, slightly uphill towards the castle from the station. On entry it's modern, cosy has tons of cool yet bright pastel and mochilato colours to accommodate your cold wintertime self. Wooden floors, warm rugs, sheepskins on the sofas, slate walls and oak barrelled tables, with slight distinct Scottish themes due to their tartan furnished accessories completes the feel.  I sat in One Lounge, had tea and food self served for well under £10, you know bran, meats cheese and croissant that sort of stuff. With a copy of local newspaper in hand I really needed to eat, stay warm and access wifi, the reasons I came in.  Some hotels in the city give you access to wifi if your a paying guest, but repeated attempts to access their wifi even with staff help, eventually gave access but in German. On going to reception again for additional help the staff were great got the correct website for me, but still no luck.  Hmmm, by the time this was all done I came back to cold tea and didn't really have time now to finish a second cup of hot Earl Grey, so sorry to staff for removing an almost full cup of tea. Great place though nice breakfast and funky decor especially the plasma tv displaying a roasting log fire in reception, but sadly for me no easy access to wifi when badly required.  Maybe next time I will endeavour to try food against technology!\n"
_str

"Had Continental style breakfast here a short distance from Waverley Station as I had half hour before a train home and wanted to try somewhere new. Motel1One is nearby and a easy two minute walk, slightly uphill towards the castle from the station. On entry it's modern, cosy has tons of cool yet bright pastel and mochilato colours to accommodate your cold wintertime self. Wooden floors, warm rugs, sheepskins on the sofas, slate walls and oak barrelled tables, with slight distinct Scottish themes due to their tartan furnished accessories completes the feel.  I sat in One Lounge, had tea and food self served for well under £10, you know bran, meats cheese and croissant that sort of stuff. With a copy of local newspaper in hand I really needed to eat, stay warm and access wifi, the reasons I came in.  Some hotels in the city give you access to wifi if your a paying guest, but repeated attempts to access their wifi even with staff help, eventually gave access but in German. On going to re